# Les décorateurs
## Décorateurs de base

In [24]:
def sandwich(garniture):
    def wrapper(*args, **kwargs):
        print("/TTTTTTTT\\")
        price = garniture(*args, **kwargs)
        print("\\________/")

        return price + 2

    return wrapper

@sandwich
def kebab(salade_tomates_oignons:bool):
    if salade_tomates_oignons:
        print("Salade tomates oignons")

    print('viande')

    return 5

@sandwich
def parisien():
    print("  Jambon ")
    print("  Beurre ")

    return 6

@sandwich
def bagnat():
    print("  niçoise ")
    print("  salade ")

    return 7



In [25]:
kebab(True)

/TTTTTTTT\
Salade tomates oignons
viande
\________/


7

In [18]:
print(parisien())

/TTTTTTTT\
  Jambon 
  Beurre 
\________/
8


In [19]:
bagnat()

/TTTTTTTT\
  niçoise 
  salade 
\________/


9

## Rappel sur les paramètres de fonction

In [76]:
def func(param1, /, param2, *, param3):
    pass

func(1, param2=2, param3=3)

## Décorateurs paramétrés

In [67]:
def sandwich(_func=None, *, hot:bool=False):
    def make_sandwich(garniture):
        def wrapper(*args, **kwargs):
            print("/TTTTTTTT\\")
            garniture(*args, **kwargs)
            print("\\________/")

            print("chaud" if hot else "froid")

        return wrapper

    if _func is None:
        return make_sandwich
    else:
        return make_sandwich(_func)

@sandwich(hot=True)
def panini():
    print('jambon')

@sandwich
def parisien():
    print("  Jambon ")
    print("  Beurre ")

    return 6

panini(True)

/TTTTTTTT\


TypeError: panini() takes 0 positional arguments but 1 was given

## Exercice 1

In [27]:
import time

def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(end - start, "secondes se sont écoulées")
        return result

    return wrapper

@time_it
def calculate():
    for x in range(1_000_000):
        y = x ** 2

calculate()

0.03767585754394531 secondes se sont écoulées


## Exercice 2

In [34]:
def count_calls(func):
    count = 0

    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        return func(*args, **kwargs)

    def get_counts():
        return count

    inner.nbcalls = get_counts

    return inner

In [35]:
@count_calls
@time_it
def some_func():
    print("func called")

print(some_func.nbcalls())
some_func()
some_func()
print(some_func.nbcalls())


0
func called
1.0013580322265625e-05 secondes se sont écoulées
func called
6.9141387939453125e-06 secondes se sont écoulées
2


Je dois avoir :
```bash
0
func called
func called
2
```

## Exercice 3

In [36]:
import time
from collections import OrderedDict

def cachable(func):

    previous = OrderedDict()

    def inner(value:int):
        if value not in previous:
            result = func(value)
            previous[value] = result
            if len(previous) > 10:
                previous.popitem(last=False)
        else:
            result = previous[value]
            previous.move_to_end(value, last=True)

        return result

    return inner

In [39]:
@time_it
@cachable
def long_call(value:int):
    time.sleep(2)
    return value**2

In [48]:
print(f"{long_call(1)=}")
print(f"{long_call(4)=}")
print(f"{long_call(5)=}")
print(f"{long_call(6)=}")
print(f"{long_call(7)=}")
print(f"{long_call(8)=}")
print(f"{long_call(9)=}")
print(f"{long_call(10)=}")
print(f"{long_call(11)=}")
print(f"{long_call(12)=}")
print(f"{long_call(2)=}")
print(f"{long_call(1)=}")


2.6226043701171875e-06 secondes se sont écoulées
long_call(1)=1
1.1920928955078125e-06 secondes se sont écoulées
long_call(4)=16
0.0 secondes se sont écoulées
long_call(5)=25
0.0 secondes se sont écoulées
long_call(6)=36
1.1920928955078125e-06 secondes se sont écoulées
long_call(7)=49
0.0 secondes se sont écoulées
long_call(8)=64
0.0 secondes se sont écoulées
long_call(9)=81
9.5367431640625e-07 secondes se sont écoulées
long_call(10)=100
0.0 secondes se sont écoulées
long_call(11)=121
3.814697265625e-06 secondes se sont écoulées
long_call(12)=144
2.005082845687866 secondes se sont écoulées
long_call(2)=4
2.002661943435669 secondes se sont écoulées
long_call(1)=1


## Exercice 4

In [51]:
def register(func):
    notifications.append(func)
    return func

notifications = []

@register
def notify_email():
    print("notification from email")

def notify_sms():
    print("notification from SMS")

@register
def notify_push():
    print("notification from push")


def send_notifications():
    for notif in notifications:
        notif()

In [52]:
send_notifications()

notification from email
notification from push


## Exercice 4 bis

In [61]:
def register(level):
    def notif_wrapper(func):
        notifications.append((func, level))
        return func
    return notif_wrapper

notifications = []

@register(level=1)
def notify_email():
    print("notification from email")

@register(level=2)
def notify_sms():
    print("notification from SMS")

@register(level=1)
def notify_push():
    print("notification from push")


def send_notifications(level=1):
    for notif, notif_level in notifications:
        if notif_level >= level:
            notif()

send_notifications(3)